##### Copyright 2020 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 图和函数简介

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/intro_to_graphs"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/intro_to_graphs.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/intro_to_graphs.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/intro_to_graphs.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## 综述
本指南深入TensorFlow和Keras的知识面，看看TensorFlow是如何工作的。如果你想立即开始学习Keras，请参阅我们的Keras指南收集。
在本指南中，您将看到TensorFlow如何允许您对代码进行简单更改以获得图形的核心内容，以及如何存储和表示图形，以及如何使用它们来加速和导出模型。
注意:对于那些只熟悉TensorFlow 1.x的同学，本指南演示了一个非常不同的图形视图。
这是一个简短的介绍;有关这些概念的完整介绍，请参阅 the tf.function guide。

### 图是什么?

在前三部分指南中，您已经看到TensorFlow正在快速顺利地运行。这意味着TensorFlow操作由Python执行，一个操作接着一个操作的执行，然后将结果返回给Python。Eager TensorFlow利用了gpu的优势，允许您在gpu和TPUs上放置变量、张量，甚至操作。它也很容易调试。

对于某些用户，您可能永远不需要或不想离开Python。

但是，在Python中运行TensorFlow一个接一个的操作会阻止大量其他可用的加速。如果你能从Python中提取出张量计算，你就能把它们做成图。

**图是包含一组操作对象，它们表示计算单位;以及张量对象，这些张量表示在操作之间流动的数据的单位。它们被定义为tf.Graph context。由于这些图是数据结构，所以可以在不使用原始Python代码的情况下保存、运行和恢复它们。

这就是一个简单的两层图在TensorBoard中显示的样子。

![a two-layer tensorflow graph](	
https://storage.cloud.google.com/tensorflow.org/images/two-layer-network.png)

### 图的好处

有了图，您就有了很大的灵活性。您可以在没有Python解释器的环境中使用TensorFlow图，比如移动应用程序、嵌入式设备和后端服务器。当从Python导出模型时，TensorFlow使用图形作为保存模型的格式。

图形也很容易优化，允许编译器做像这样的转换:

* 通过在计算中折叠常数节点(“常数折叠”)来静态地推断张量的值。
* 计算的独立子部分，并在线程或设备之间分割它们。
* 通过消除公共子表达式来简化算术运算。


有一个完整的优化系统，Grappler，执行这个和其他加速。

简而言之，图形非常有用，它可以让您的TensorFlow快速运行、并行运行并在多个设备上高效运行。

但是，为了方便起见，您仍然希望用Python定义我们的机器学习模型(或其他计算)，然后在需要时自动构造图。

## 跟踪图（Tracing graphs）

在TensorFlow中创建图形的方法是使用tf.function，可以作为直接调用，也可以作为装饰器。

In [2]:
import tensorflow as tf
import timeit
from datetime import datetime

In [3]:
# Define a Python function
def function_to_get_faster(x, y, b):
  x = tf.matmul(x, y)
  x = x + b
  return x

# Create a `Function` object that contains a graph
a_function_that_uses_a_graph = tf.function(function_to_get_faster)

# Make some tensors
x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

# It just works!
a_function_that_uses_a_graph(x1, y1, b1).numpy()

array([[12.]], dtype=float32)

`tf.function`-ized 函数是Python可调用函数，其工作原理与Python的等价函数相同。它们有一个特定的类(python.eager.def_function.Function)，但对您来说，它们就像未跟踪版本一样。

`tf.function` 递归地跟踪它调用的任何Python函数。

In [4]:
def inner_function(x, y, b):
  x = tf.matmul(x, y)
  x = x + b
  return x

# Use the decorator
@tf.function
def outer_function(x):
  y = tf.constant([[2.0], [3.0]])
  b = tf.constant(4.0)

  return inner_function(x, y, b)

# Note that the callable will create a graph that
# includes inner_function() as well as outer_function()
outer_function(tf.constant([[1.0, 2.0]])).numpy()

array([[12.]], dtype=float32)

如果你用过TensorFlow 1.x，您将注意到在任何时候都不需要定义占位符或tf.Sesssion。

### 流控制和副作用（Flow control and side effects）

流量控制和回路通常通过默认方法tf.autograph转换为张力流。Autograph使用了一组方法，包括标准化循环构造、展开和AST操作。

In [5]:
def my_function(x):
  if tf.reduce_sum(x) <= 1:
    return x * x
  else:
    return x-1

a_function = tf.function(my_function)

print("First branch, with graph:", a_function(tf.constant(1.0)).numpy())
print("Second branch, with graph:", a_function(tf.constant([5.0, 5.0])).numpy())

First branch, with graph: 1.0
Second branch, with graph: [4. 4.]


您可以直接调用Autograph转换来查看Python是如何转换为TensorFlow操作的。  这大部分是不可读的，但您可以看到转换。

In [6]:
# Don't read the output too carefully.
print(tf.autograph.to_code(my_function))

def tf__my_function(x):
    with ag__.FunctionScope('my_function', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (retval_, do_return)

        def set_state(vars_):
            nonlocal retval_, do_return
            (retval_, do_return) = vars_

        def if_body():
            nonlocal retval_, do_return
            try:
                do_return = True
                retval_ = (ag__.ld(x) * ag__.ld(x))
            except:
                do_return = False
                raise

        def else_body():
            nonlocal retval_, do_return
            try:
                do_return = True
                retval_ = (ag__.ld(x) - 1)
            except:
                do_return = False
                raise
        ag__.if_stmt((ag__.converted_call(ag__.ld(tf).reduce_sum, (ag

Autograph自动转换if-then子句、循环、break、return、continue等。

大多数的时候, Autograph工作的时候不会有特别的考虑。然而有些说明和tf.function guide以及tf.function guide可以帮助到你。

### 看到速度的加快（Seeing the speed up）

在tf.function只是包装一个使用张量的函数并不能加速代码的运行，对于在一台机器上多次调用的小函数，调用一个图形或图形片段的开销可能会占用大多数运行时间。此外，如果大部分计算已经在加速器上进行，比如大量的gpu卷积，图形加速就不会很明显。

对于复杂的计算，图形可以提供显著的加速。这是因为图减少了python与设备的通信，并执行了一些加速操作。

这段代码在一些密集层上运行了几次。

In [7]:
# Create an oveerride model to classify pictures
class SequentialModel(tf.keras.Model):
  def __init__(self, **kwargs):
    super(SequentialModel, self).__init__(**kwargs)
    self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
    self.dense_1 = tf.keras.layers.Dense(128, activation="relu")
    self.dropout = tf.keras.layers.Dropout(0.2)
    self.dense_2 = tf.keras.layers.Dense(10)

  def call(self, x):
    x = self.flatten(x)
    x = self.dense_1(x)
    x = self.dropout(x)
    x = self.dense_2(x)
    return x

input_data = tf.random.uniform([60, 28, 28])

eager_model = SequentialModel()
graph_model = tf.function(eager_model)

print("Eager time:", timeit.timeit(lambda: eager_model(input_data), number=10000))
print("Graph time:", timeit.timeit(lambda: graph_model(input_data), number=10000))


Eager time: 5.330957799000316


Graph time: 3.0375442899999143


### 多态函数（Polymorphic functions）

跟踪一个函数时，将创建一个多态的函数对象。多态函数是一种Python可调用函数，它在一个API后面封装了几个具体的函数图。

您可以在所有不同类型的dtype和形状上使用此函数。每次使用新的参数签名调用它时，原始函数将使用新的参数重新跟踪。  然后该函数存储tf.graph对应于concrete_function中的跟踪。如果函数已经被这种参数跟踪过了，你就得到了你的预跟踪图。

从概念上讲,然后:
* 一个tf.Graph是描述计算的原始的、可移植的数据结构
* 函数是一个缓存，跟踪，调度程序在ConcreteFunctions之上
* ConcreteFunction是一个与eager兼容的图形包装器，它允许您从Python执行图形



### 检查多态函数（Inspecting polymorphic functions）

您可以检查a_function，它是在Python函数的my_function中调用tf.function的结果。在本例中，使用三种参数调用a_function会产生三个不同的具体函数。

In [8]:
print(a_function)

print("Calling a `Function`:")
print("Int:", a_function(tf.constant(2)))
print("Float:", a_function(tf.constant(2.0)))
print("Rank-1 tensor of floats", a_function(tf.constant([2.0, 2.0, 2.0])))

Calling a `Function`:
Int: tf.Tensor(1, shape=(), dtype=int32)
Float: tf.Tensor(1.0, shape=(), dtype=float32)
Rank-1 tensor of floats tf.Tensor([1. 1. 1.], shape=(3,), dtype=float32)


In [9]:
# Get the concrete function that works on floats
print("Inspecting concrete functions")
print("Concrete function for float:")
print(a_function.get_concrete_function(tf.TensorSpec(shape=[], dtype=tf.float32)))
print("Concrete function for tensor of floats:")
print(a_function.get_concrete_function(tf.constant([2.0, 2.0, 2.0])))


Inspecting concrete functions
Concrete function for float:
ConcreteFunction my_function(x)
  Args:
    x: float32 Tensor, shape=()
  Returns:
    float32 Tensor, shape=()
Concrete function for tensor of floats:
ConcreteFunction my_function(x)
  Args:
    x: float32 Tensor, shape=(3,)
  Returns:
    float32 Tensor, shape=(3,)


In [10]:
# Concrete functions are callable
# Note: You won't normally do this, but instead just call the containing `Function`
cf = a_function.get_concrete_function(tf.constant(2))
print("Directly calling a concrete function:", cf(tf.constant(2)))

Directly calling a concrete function: tf.Tensor(1, shape=(), dtype=int32)


在这个示例中，您看到的是堆栈中相当深入的部分。除非您专门管理跟踪，否则通常不需要直接调用此处所示的具体函数

## 重新回到即可执行（Reverting to eager execution）

您可能会发现自己正在查看长堆栈跟踪，特别是那些引用tf.Graph()或tf.Graph().as_default()。这意味着您可能是在一个图形上下文中运行。TensorFlow中的核心函数使用图形上下文，比如Keras的model.fit()。

调试即刻执行通常要容易得多。堆栈跟踪应该相对简短并且容易理解。

在图使调试变得棘手的情况下，您可以恢复到使用急执行进行调试。

这里有一些方法，你可以确保你可以快速地运行:

* 直接将模型和层作为可调用对象调用

* 当使用Keras compile/fit时，在编译时使用model.compile(run_gree= True)

* 通过 tf.config.run_functions_eagerly(True)设置全局执行模式


### Using `run_eagerly=True`

In [11]:
# Define an identity layer with an eager side effect
class EagerLayer(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super(EagerLayer, self).__init__(**kwargs)
    # Do some kind of initialization here

  def call(self, inputs):
    print("\nCurrently running eagerly", str(datetime.now()))
    return inputs

In [12]:
# Create an override model to classify pictures, adding the custom layer
class SequentialModel(tf.keras.Model):
  def __init__(self):
    super(SequentialModel, self).__init__()
    self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
    self.dense_1 = tf.keras.layers.Dense(128, activation="relu")
    self.dropout = tf.keras.layers.Dropout(0.2)
    self.dense_2 = tf.keras.layers.Dense(10)
    self.eager = EagerLayer()

  def call(self, x):
    x = self.flatten(x)
    x = self.dense_1(x)
    x = self.dropout(x)
    x = self.dense_2(x)
    return self.eager(x)

# Create an instance of this model
model = SequentialModel()

# Generate some nonsense pictures and labels
input_data = tf.random.uniform([60, 28, 28])
labels = tf.random.uniform([60])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

首先，无需急于编译模型。注意，模型没有被跟踪;尽管它的命名和编译只设置损失函数、优化和其他训练参数。

In [13]:
model.compile(run_eagerly=False, loss=loss_fn)

现在，调用fit并看到该函数被跟踪(两次)，然后即可效果将不再运行。

In [14]:
model.fit(input_data, labels, epochs=3)

Epoch 1/3



Currently running eagerly 2020-11-04 02:22:28.114630



Currently running eagerly 2020-11-04 02:22:28.233822


1/2 [==============>...............] - ETA: 0s - loss: 1.8508

2/2 [==============================] - 0s 2ms/step - loss: 0.9890


Epoch 2/3
1/2 [==============>...............] - ETA: 0s - loss: 0.0020

2/2 [==============================] - 0s 1ms/step - loss: 0.0017


Epoch 3/3
1/2 [==============>...............] - ETA: 0s - loss: 7.8775e-04

2/2 [==============================] - 0s 1ms/step - loss: 5.4478e-04


If you run even a single epoch in eager, however, you can see the eager side effect twice.

In [15]:
print("Running eagerly")
# When compiling the model, set it to run eagerly
model.compile(run_eagerly=True, loss=loss_fn)

model.fit(input_data, labels, epochs=1)


Running eagerly



Currently running eagerly 2020-11-04 02:22:28.449835
1/2 [==============>...............] - ETA: 0s - loss: 7.8712e-04


Currently running eagerly 2020-11-04 02:22:28.472645
2/2 [==============================] - 0s 6ms/step - loss: 4.1988e-04


### Using `run_functions_eagerly`
你也可以在全局范围内设置每一项都可以快速运行。这是一个绕过多态函数的跟踪函数并直接调用原始函数的开关。您可以使用它进行调试。

In [16]:
# Now, globally set everything to run eagerly
tf.config.run_functions_eagerly(True)
print("Run all functions eagerly.")

# Create a polymorphic function
polymorphic_function = tf.function(model)

print("Tracing")
# This does, in fact, trace the function
print(polymorphic_function.get_concrete_function(input_data))

print("\nCalling twice eagerly")
# When you run the function again, you will see the side effect
# twice, as the function is running eagerly.
result = polymorphic_function(input_data)
result = polymorphic_function(input_data)

Run all functions eagerly.
Tracing

Currently running eagerly 2020-11-04 02:22:28.502694
ConcreteFunction function(self)
  Args:
    self: float32 Tensor, shape=(60, 28, 28)
  Returns:
    float32 Tensor, shape=(60, 10)

Calling twice eagerly

Currently running eagerly 2020-11-04 02:22:28.507036

Currently running eagerly 2020-11-04 02:22:28.508331


In [17]:
# Don't forget to set it back when you are done
tf.config.experimental_run_functions_eagerly(False)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


## 跟踪和性能（Tracing and performance）

跟踪会带来一些开销。虽然跟踪小功能很快，但大型模型可能需要明显的长时间来跟踪。这种投资通常会很快得到回报，带来性能提升，但要意识到，任何大型模型训练的前几个阶段都可能由于跟踪而变慢。

无论您的模型有多大，您都希望避免频繁跟踪。 这部分的tf的函数指南讨论了如何设置输入规格和使用张量参数来避免回溯。 如果您发现性能异常差，最好检查一下是否不小心进行了重跟踪。

您可以添加一个eager-only的副作用(比如打印Python参数)，这样就可以看到跟踪函数的时间。在这里，您可以看到额外的回溯，因为新的Python参数总是触发回溯。

In [18]:
# Use @tf.function decorator
@tf.function
def a_function_with_python_side_effect(x):
  print("Tracing!")  # This eager
  return x * x + tf.constant(2)

# This is traced the first time
print(a_function_with_python_side_effect(tf.constant(2)))
# The second time through, you won't see the side effect
print(a_function_with_python_side_effect(tf.constant(3)))

# This retraces each time the Python argument changes,
# as a Python argument could be an epoch count or other
# hyperparameter
print(a_function_with_python_side_effect(2))
print(a_function_with_python_side_effect(3))


Tracing!
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(11, shape=(), dtype=int32)
Tracing!
tf.Tensor(6, shape=(), dtype=int32)
Tracing!
tf.Tensor(11, shape=(), dtype=int32)


## 下一步（Next steps）

你可以在“tf.function的API参考页面和[指南]”上进行更深入的阅读。